In [2]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
  
#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity
  
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [38]:
validation_data = pd.read_excel('validation_data.xlsx')
validation_data = validation_data[['Sentence','Sentiment (Engine Assigned)','Manual Verification','Corrected Sentiments']]
validation_data

c:\users\archana\anaconda3\envs\social-env\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Sentence,Sentiment (Engine Assigned),Manual Verification,Corrected Sentiments
0,#mahindra_suv#thar#mahindra_thar @anandmahindr...,Neutral,Correct,NaN
1,sa sreejith ss mahindra #withyouhmesha,Neutral,Correct,NaN
2,he is very helpful and explain nature of job a...,Positive,Correct,NaN
3,thanks mahindra,Positive,Correct,NaN
4,mujhe eha ka service sacha lga main service se...,Neutral,Incorrect,Positive
...,...,...,...,...
95,excellent service experience in pratik automob...,Positive,Correct,NaN
96,good quality car serves mahindra marazzo m6,Positive,Correct,NaN
97,excellent service experience at sunshine autos...,Positive,Correct,NaN
98,when i'm going for top variant of creta petrol...,Neutral,Correct,NaN


In [46]:
#Vader Sentiment
validation_data['vader_score'] = validation_data['Sentence'].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])
validation_data['vader_sentiment'] = validation_data['vader_score'].apply(lambda x: 'Positive' if x > 0 else ('Neutral' if x == 0 else 'Negative'))

In [47]:
#textblob sentiment
validation_data['TextBlob_Subjectivity'] =  validation_data['Sentence'].apply(getSubjectivity)
validation_data['TextBlob_Polarity'] = validation_data['Sentence'].apply(getPolarity)
validation_data['TextBlob_Sentiment'] = validation_data['TextBlob_Polarity'].apply(getAnalysis)

In [48]:
validation_data

,Sentence,Sentiment (Engine Assigned),Manual Verification,Corrected Sentiments,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Sentiment,vader_score,vader_sentiment
0,#mahindra_suv#thar#mahindra_thar @anandmahindr...,Neutral,Correct,NaN,0.000000,0.000000,Neutral,0.0000,Neutral
1,sa sreejith ss mahindra #withyouhmesha,Neutral,Correct,NaN,0.000000,0.000000,Neutral,0.0000,Neutral
2,he is very helpful and explain nature of job a...,Positive,Correct,NaN,0.650000,0.350000,Positive,0.7328,Positive
3,thanks mahindra,Positive,Correct,NaN,0.200000,0.200000,Positive,0.4404,Positive
4,mujhe eha ka service sacha lga main service se...,Neutral,Incorrect,Positive,0.333333,0.166667,Positive,0.0000,Neutral
...,...,...,...,...,...,...,...,...,...
95,excellent service experience in pratik automob...,Positive,Correct,NaN,0.933333,0.800000,Positive,0.8934,Positive
96,good quality car serves mahindra marazzo m6,Positive,Correct,NaN,0.600000,0.700000,Positive,0.4404,Positive
97,excellent service experience at sunshine autos...,Positive,Correct,NaN,0.600000,0.600000,Positive,0.8689,Positive
98,when i'm going for top variant of creta petrol...,Neutral,Correct,NaN,0.500000,0.500000,Positive,0.2023,Positive


In [49]:
validation_data.columns

Index(['Sentence', 'Sentiment (Engine Assigned)', 'Manual Verification',
       'Corrected Sentiments', 'TextBlob_Subjectivity', 'TextBlob_Polarity',
       'TextBlob_Sentiment', 'vader_score', 'vader_sentiment'],
      dtype='object')

In [50]:
validation_data['vader_correctness'] = validation_data.apply(lambda x: x['Sentiment (Engine Assigned)'].lower() == x.vader_sentiment.lower() if x['Manual Verification'] == 'Correct' else x['Corrected Sentiments'].lower() == x.vader_sentiment.lower(), axis=1)

In [51]:
validation_data['vader_correctness'].value_counts()

True     90
False    10
Name: vader_correctness, dtype: int64

In [52]:
validation_data['textblob_correctness'] = validation_data.apply(lambda x: x['Sentiment (Engine Assigned)'].lower() == x.TextBlob_Sentiment.lower() if x['Manual Verification'] == 'Correct' else x['Corrected Sentiments'].lower() == x.TextBlob_Sentiment.lower(), axis=1)

In [53]:
validation_data['textblob_correctness'].value_counts()

True     86
False    14
Name: textblob_correctness, dtype: int64

In [54]:
validation_data['Manual Verification'].value_counts()

Correct      95
Incorrect     5
Name: Manual Verification, dtype: int64

In [55]:
validation_data.to_excel('sentiment_validation.xlsx')

In [4]:
%%time
df1 = pd.read_excel('Social_Listening_Excel_SQL_JAN17_AUG20.xlsx')
df2 = pd.read_excel('SOCIAL_LISTENING_EXCEL_SQL_SEP`20_MAY`21.xlsx')
df = pd.concat([df1[['Brand', 'Sentiment', 'Content']], df1[['Brand', 'Sentiment', 'Content']]], ignore_index=True)
df.shape

Wall time: 3min 19s


(999486, 3)

In [5]:
df.head()

,Brand,Sentiment,Content
0,Scorpio,Neutral,"16 hours ago - First introduced in 2007, the M..."
1,Scorpio,Neutral,1 day ago - CarWale - Used Mahindra Scorpio [2...
2,Scorpio,Neutral,20 hours ago - CarWale - Used Mahindra Scorpio...
3,Scorpio,Neutral,1 hour ago - The 2017 Mahindra Scorpio Getaway...
4,Scorpio,Neutral,19 hours ago - CarWale - Used Mahindra Scorpio...


In [6]:
def preprocess_df(df):
    df = df.copy()
    df = df[~df['Content'].isna()]
    df.drop_duplicates(subset=['Content'], inplace=True)
    return df

In [7]:
df[~df['Content'].isna()].shape

(996706, 3)

In [8]:
df[~df['Content'].isna()].Content.nunique()

302462

In [10]:
processed_df = preprocess_df(df)
processed_df.shape

(302462, 3)

In [11]:
def preprocess_text(text):
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    # it will remove hashtags. We have to be careful here not to remove 
    # the whole hashtag because text of hashtags contains huge information. 
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)
    # it will remove single numeric terms in the text. 
    text = re.sub(r'[0-9]', '', text)
    # remove 
    text = re.sub('@[^\s]+', '', text)
    text = re.sub(r'day ago - |hours ago - |hour ago|CarWale', '', text)
    return text

In [19]:
# preprocess_text('@anandmahindra how are you doing https://t.co/dngwphbRES. ')

In [12]:
%%time
processed_df['processed_content'] = processed_df['Content'].apply(lambda x: preprocess_text(str(x)))

Wall time: 2.65 s


In [13]:
processed_df.head()

,Brand,Sentiment,Content,processed_content
0,Scorpio,Neutral,"16 hours ago - First introduced in 2007, the M...","First introduced in , the Mahindra Getaway is..."
1,Scorpio,Neutral,1 day ago - CarWale - Used Mahindra Scorpio [2...,- Used Mahindra Scorpio [-] Ex for sale in H...
2,Scorpio,Neutral,20 hours ago - CarWale - Used Mahindra Scorpio...,- Used Mahindra Scorpio [-] Ex for sale in H...
3,Scorpio,Neutral,1 hour ago - The 2017 Mahindra Scorpio Getaway...,- The Mahindra Scorpio Getaway has been spo...
4,Scorpio,Neutral,19 hours ago - CarWale - Used Mahindra Scorpio...,- Used Mahindra Scorpio [-] VLX WD BS-III fo...


In [14]:
#Vader Sentiment
processed_df['vader_score'] = processed_df['processed_content'].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])
processed_df['vader_sentiment'] = processed_df['vader_score'].apply(lambda x: 'Positive' if x > 0 else ('Neutral' if x == 0 else 'Negative'))

In [15]:
#textblob sentiment
processed_df['TextBlob_Subjectivity'] =  processed_df['processed_content'].apply(getSubjectivity)
processed_df['TextBlob_Polarity'] = processed_df['processed_content'].apply(getPolarity)
processed_df['TextBlob_Sentiment'] = processed_df['TextBlob_Polarity'].apply(getAnalysis)

In [16]:
processed_df

,Brand,Sentiment,Content,processed_content,vader_score,vader_sentiment,TextBlob_Subjectivity,TextBlob_Polarity,TextBlob_Sentiment
0,Scorpio,Neutral,"16 hours ago - First introduced in 2007, the M...","First introduced in , the Mahindra Getaway is...",0.3612,Positive,0.202083,-0.01875,Negative
1,Scorpio,Neutral,1 day ago - CarWale - Used Mahindra Scorpio [2...,- Used Mahindra Scorpio [-] Ex for sale in H...,0.0772,Positive,0.000000,0.00000,Neutral
2,Scorpio,Neutral,20 hours ago - CarWale - Used Mahindra Scorpio...,- Used Mahindra Scorpio [-] Ex for sale in H...,0.0772,Positive,0.000000,0.00000,Neutral
3,Scorpio,Neutral,1 hour ago - The 2017 Mahindra Scorpio Getaway...,- The Mahindra Scorpio Getaway has been spo...,0.0000,Neutral,0.000000,0.00000,Neutral
4,Scorpio,Neutral,19 hours ago - CarWale - Used Mahindra Scorpio...,- Used Mahindra Scorpio [-] VLX WD BS-III fo...,0.6633,Positive,0.000000,0.00000,Neutral
...,...,...,...,...,...,...,...,...,...
499738,Marazzo,Neutral,@anandmahindra https://t.co/dngwphbRES. The p...,,0.0000,Neutral,0.000000,0.00000,Neutral
499739,Marazzo,Neutral,@anandmahindra सर आप यह गाड़ी महान है अपने देश...,सर आप यह गाड़ी महान है अपने देश को अच्छा तोहफ...,0.3182,Positive,0.000000,0.00000,Neutral
499740,Marazzo,Neutral,Mahindra Marazzo review. The shark inspired MU...,Mahindra Marazzo review. The shark inspired MU...,0.7345,Positive,0.550000,0.43750,Positive
499741,Marazzo,Neutral,Mahindra cars price list in India *All prices ...,Mahindra cars price list in India *All prices ...,0.8000,Positive,0.488889,0.50000,Positive


In [78]:
processed_df['Content'][0]

'16 hours ago - First introduced in 2007, the Mahindra Getaway is due for an upgrade. The Scorpio-based pick-up is sold as Pik-Up in a few international markets like Australia#160;...'

In [17]:
processed_df.to_excel('result.xlsx', index=False)